In [ ]:
import os
import cv2

import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

import torch
import torch.nn as nn

# The cross-entropy loss penalizes the model more when it is more confident in the incorrect class
# which makes intuitive sense.
from torch.nn import CrossEntropyLoss

# Adam is an optimization algorithm that can be used instead of the classical SGD procedure
# to update network weights iterative based in training data.
from torch.optim import Adam , lr_scheduler
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.io import read_image

from torchinfo import summary

# np.random.seed(0)
# torch.manual_seed(0)

In [ ]:
# torch.cuda.current_device()
# torch.cuda.get_device_name(0)
# torch.cuda.device_count()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Torch is using device:', device)
torch.cuda.get_device_name(device)
# start= torch.cuda.Event(enable_timing=True)
# end=torch.cuda.Event(enable_timing=True)

In [ ]:
path_tr = '../../data/CIFAK/train'
path_val = '../../data/CIFAK/test'

#### Remove Spaces from the Dataset folder to simplify reading from it

In [ ]:
# def remove_fname_space(path):
#     for filename in os.listdir(path):
#         my_source = path + "/" + filename
#         my_dest = path + "/" + filename.strip().replace(" ", "")
#         os.rename(my_source, my_dest)


# remove_fname_space(path_tr + "/REAL")
# remove_fname_space(path_tr + "/FAKE")
# remove_fname_space(path_val + "/REAL")
# remove_fname_space(path_val + "/FAKE")

#### Gather all the data from the "CIFAK" dataset to be used later

In [ ]:
def dataset_data(img_path, folder_name, img_class):
    fname = os.listdir(img_path + "/"+folder_name)
    fname.sort()
    fpath = [img_path + "/"+folder_name+"/" + f for f in fname]
    # print(fpath[0])
    labels = [img_class]*len(fname)
    return fpath, labels

In [ ]:
fpath_tr_real, labels_tr_real = dataset_data(path_tr, "REAL", 1.0)
fpath_tr_fake, labels_tr_fake = dataset_data(path_tr, "FAKE", 0.0)
fpath_tr = fpath_tr_real + fpath_tr_fake
labels_tr = labels_tr_real+labels_tr_fake
tr_dict = {'Image_path': fpath_tr, 'True?': labels_tr}
tr_df = pd.DataFrame(tr_dict)
tr_df.to_csv(path_tr+"/tr_annotation.csv")


fpath_val_real, labels_val_real = dataset_data(path_val, "REAL", 1.0)
fpath_val_fake, labels_val_fake = dataset_data(path_val, "FAKE", 0.0)
fpath_val = fpath_val_real + fpath_val_fake
labels_val = labels_val_real+labels_val_fake
val_dict = {'Image_path': fpath_val, 'True?': labels_val}
val_df = pd.DataFrame(val_dict)
val_df.to_csv(path_val+"/val_annotation.csv")

print("")
print("Total Number of:          Real         Fake         Total")
print("------------------------------------------------------------")
print("Training Samples:        ",len(fpath_tr_real),"     ",len(fpath_tr_fake),"       ",len(fpath_tr))
print("Validation Samples:      ",len(fpath_val_real),"     ",len(fpath_val_fake),"       ",len(fpath_val))

In [ ]:
print("tr_df")
tr_df.iloc[[0,1,99998,99999]]

In [ ]:
print("val_df")
val_df.iloc[[0,1,19998,19999]]

#### Determine the number of slices (patches) of the image

In [ ]:
Img_horizontal_slices= 4
images_batch = 32

In [ ]:
img_shape = cv2.imread(fpath_tr[0]).shape
print("Image Shape: ",img_shape)

slice_width = img_shape[0]//Img_horizontal_slices
print("Image Slice Shape: ",(slice_width,slice_width))

total_img_slices = Img_horizontal_slices**2
print("Total Number of Slices per Image: ",total_img_slices)
print("Total Number of Images per Loader: ",images_batch)


In [ ]:
class SliceImage:
    def __init__(self, slice_width):
        self.slice_width = slice_width

    def slice(self, img):
        # img: is a tensor of the shape (Color_Channels x Rows (Hight) x Columns (Width))
        #
        # Make a slice every "slice_width" as we are moving across dimension 1 (as we are moving
        # vertically across rows)
        img = img.unfold(1, self.slice_width, self.slice_width)
        # Make a slice every slice_width as we are moving across dimension 2,
        # Note that previous operation has added new dimension at the beginning
        # refers to no. of vertical slices, hence 2 here still refers to the rows.
        img = img.unfold(2, self.slice_width, self.slice_width)
        return img

    def __call__(self, img):
        img = self.slice(img)
        channels = img.size(0)

        return img.reshape(-1, self.slice_width * self.slice_width * channels)

    plt.show()

In [ ]:
# Define the transforms on the input data (x) tensor
data_transform = transforms.Compose([
    transforms.ToPILImage(),
    # transforms.Resize((32, 32)),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    SliceImage(slice_width=slice_width)
])

In [ ]:
# Custom Dataset Class
class Images_Dataset(Dataset):
    def __init__(self, annotations_file, transform=None):
        
        self.annotation = pd.read_csv(annotations_file)
        self.transform = transform

    def __getitem__(self, index):
        
        img = read_image(self.annotation.iloc[index, 1])
        # img.to(device)
        # labels = torch.tensor(self.annotation.iloc[index, 2]).unsqueeze(1)
        labels = torch.tensor(self.annotation.iloc[index, 2])

        # print(type(labels))
        # print(labels.size())

        if self.transform:
            img_t = self.transform(img)

        return img, img_t, labels

    def __len__(self):
        # To return the length of the dataset
        return self.annotation.shape[0]

In [ ]:
# Define the dataset object and configure the Data Loader
tr_annotation_file = path_tr+"/tr_annotation.csv"
val_annotation_file = path_val+"/val_annotation.csv"

Im_tr_dataset = Images_Dataset(tr_annotation_file, data_transform)

Im_tr_loader = DataLoader(dataset=Im_tr_dataset,
                          batch_size=images_batch,
                          # Drops the last mini batch if less than the batch size (could enhance the model accuracy)
                          drop_last=True,
                          shuffle=True,
                          num_workers=4)


Im_val_dataset = Images_Dataset(val_annotation_file, data_transform)

Im_val_loader = DataLoader(dataset=Im_val_dataset,
                          batch_size=images_batch,
                          # Drops the last mini batch if less than the batch size (could enhance the model accuracy)
                          drop_last=True,
                          shuffle=True,
                          num_workers=4)

#### Plot the original Image and the slices Image

In [ ]:
def img_data(Img_train_loader, index):

    date_sample=Img_train_loader.dataset[index]

    img = date_sample[0]
    img_t = date_sample[1]
    label = date_sample[2]

    channels = img.size(0)
    img_size = img.size(1)

    total_img_slices = img_t.size(0)
    slice_flat = img_t.size(1)

    imgs_per_batch = Im_tr_loader.batch_size

    slice_width = int(np.sqrt(slice_flat/channels))

    return img, channels, img_size, slice_width, total_img_slices, slice_flat, imgs_per_batch, label

In [ ]:
def img_plot(img, slice_width):
    slice = SliceImage(slice_width)
    sliced_img = slice.slice(img).permute(1, 2, 0, 3, 4)

    fig = plt.figure(layout="constrained", figsize=(4, 4))
    subfigs = fig.subfigures(2, 1,height_ratios=[1., 1.])
    
    axs0 = subfigs[0].subplots(1, 1)
    axs0.imshow(img.permute(1, 2, 0))
    axs0.axis('off')

    grid = ImageGrid(subfigs[1], 111, nrows_ncols=(
        sliced_img.size(0), sliced_img.size(1)), axes_pad=0.03)

    for i, ax in enumerate(grid):
        i_b4 = str(np.base_repr(i, sliced_img.size(0))).zfill(2)
        row = int(i_b4[0])
        column = int(i_b4[1])
        patch = sliced_img[row][column].permute(1, 2, 0).numpy()
        ax.imshow(patch)
        ax.axis('off')

In [ ]:
img, ch, img_size, slice_width, slices, slice_flat, batch_size, label = img_data(
    Im_tr_loader, 2)
img_plot(img, slice_width)
# torch.tensor().size()

### Build the VIT Model : Embedding &rarr; Transformer Encoder &rarr; MLP_Head

In [ ]:
class ImageEmbedding(nn.Module):

    def __init__(self, slice_input_size, slice_embed_size, img_slices, dropout_ratio=0.2):
        super().__init__()

        self.img_to_embed = nn.Linear(slice_input_size, slice_embed_size)
        self.cls_to_embed = nn.Parameter(torch.rand(1, slice_embed_size))
        self.pos_to_embed = nn.Parameter(
            torch.rand(1, img_slices + 1, slice_embed_size))

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, flattened_imgs):
        # the input is a batch of images each has been sliced to patches and each slice
        # has been flattened. i.e. the shape of the input is:
        # [no. of images ,no. of slices per image, size of the flattened image slice]
        #
        img_embedding = self.img_to_embed(flattened_imgs)

        cls_embedding = self.cls_to_embed.repeat(img_embedding.size(0), 1, 1)
        img_embedding = torch.concat([cls_embedding, img_embedding], dim=1)

        position_embedding = self.pos_to_embed.repeat(
            img_embedding.size(0), 1, 1)

        img_and_pos_embedding = img_embedding + position_embedding
        embedding = self.dropout(img_and_pos_embedding)
        return embedding

In [ ]:
_, tr_img_t, _ = next(iter(Im_tr_loader))

slice_embed = slice_flat

embedding_layer = ImageEmbedding(slice_input_size=slice_flat,
                                 slice_embed_size=slice_embed,
                                 img_slices=slices,
                                 dropout_ratio=0.2)

embedding_output = embedding_layer(tr_img_t)

print("Input Shape:  ", tr_img_t.size())
print("Output Shape: ", embedding_output.size())
# print(embedding_output)

summary(model=embedding_layer,
        input_size=tr_img_t.size(),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
class Encoder(nn.Module):

    def __init__(self, size, num_heads, dropout=0.1):
        super().__init__()

        self.norm_attention = nn.LayerNorm(size)
        self.query = nn.Linear(size, size)
        self.key = nn.Linear(size, size)
        self.value = nn.Linear(size, size)
        self.attention = nn.MultiheadAttention(
            embed_dim=size,
            num_heads=num_heads,
            dropout=0.0,
            bias=True,
            batch_first=True)

        self.norm_feed_forward = nn.LayerNorm(size)
        self.feed_forward = nn.Sequential(
            nn.Linear(size, 4 * size),
            nn.Dropout(dropout),
            nn.GELU(),
            nn.Linear(4 * size, size),
            nn.Dropout(dropout))

    def forward(self, input_tensor):
        attn_input = self.norm_attention(input_tensor)
        query = self.query(attn_input)
        key = self.key(attn_input)
        value = self.value(attn_input)
        attn_output, _ = self.attention(query=query, key=key, value=value)

        attn_plus_norm = input_tensor + attn_output

        mlp_input = self.norm_feed_forward(attn_plus_norm)
        output = attn_plus_norm + self.feed_forward(mlp_input)

        return output

In [ ]:
encoder_layer = Encoder(size=slice_flat, num_heads=12, dropout=0.1)
encoder_output = encoder_layer(embedding_output)

print("Output Shape: ", encoder_output.size())

summary(model=encoder_layer,
        input_size=embedding_output.size(),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


In [ ]:
class VIT(nn.Module):
    def __init__(self,
                 slice_input_size,
                 slice_embed_size,
                 img_slices,
                 num_classes,
                 num_heads,
                 num_encoders=1,
                 emb_dropout=0.1,
                 enc_dropout=0.1,
                 lr=1e-4, min_lr=4e-5,
                 weight_decay=0.1,
                 epochs=200):
        super().__init__()

        self.embedding = ImageEmbedding(
            slice_input_size=slice_input_size,
            slice_embed_size=slice_embed_size,
            img_slices=img_slices,
            dropout_ratio=emb_dropout)
        
        self.encoders = nn.Sequential(
            *[Encoder(slice_embed_size, num_heads, dropout=enc_dropout) for _ in range(num_encoders)],)
        
        self.mlp_head = nn.Linear(slice_embed_size, num_classes)
        self.sigmoid=nn.Sigmoid()

        self.lr = lr
        self.min_lr = min_lr
        self.weight_decay = weight_decay
        self.epochs = epochs

    def forward(self, flattened_img):
        emb = self.embedding(flattened_img)
        attn = self.encoders(emb)
        output = torch.round(self.sigmoid(self.mlp_head(attn[:, 0, :])))

        return output

In [ ]:
slice_embed = slice_flat*4

vit_model = VIT(slice_input_size=slice_flat,
                slice_embed_size=slice_embed,
                img_slices=slices,
                num_classes=1,
                num_heads=12,
                num_encoders=2,
                emb_dropout=0.1,
                enc_dropout=0.1,
                lr=1e-4, min_lr=4e-5,
                weight_decay=0.1,
                epochs=200)

vit_output = vit_model(tr_img_t)

print("Input Shape: ", tr_img_t.size())
print("Output Shape: ", vit_output.size())

summary(model=vit_model,
        input_size=tr_img_t.size(),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
vit_epochs = 30         # Number of epochs
base_lr = 10e-3         # Base LR
weight_decay = 0.03     # Weight decay for ViT-Base (on ImageNet-21k)

In [ ]:
# Betas used for Adam in paper are 0.9 and 0.999, which are the default in PyTorch
vit_optimizer = Adam(vit_model.parameters(), lr=base_lr, weight_decay=weight_decay)
criterion = CrossEntropyLoss() # returns the mean loss for the batch
scheduler = lr_scheduler.LinearLR(vit_optimizer)

In [ ]:
vit_model.to(device)

for epoch in range(vit_epochs):

    # Training Loop
    vit_model.train()

    accumulated_tr_loss = 0
    accumulated_tr_accuracy = 0
    accumulated_val_loss = 0
    accumulated_val_accuracy = 0

    tr_loss_history = []
    tr_accuracy_history = []
    val_loss_history = []
    val_accuracy_history = []

    loop = tqdm(enumerate(Im_tr_loader), total=len(Im_tr_loader))
    for batch, (_, inputs, targets) in loop:

        # Put inputs and labels in device cuda
        tr_images = inputs.to(device)
        tr_labels = targets.to(device)

        # Forward Path
        tr_outputs = vit_model(tr_images).squeeze(1)

        loss = criterion(tr_outputs,tr_labels)

        accumulated_tr_loss += loss.item()
        accumulated_tr_accuracy += (tr_labels == tr_outputs).sum().item()/tr_labels.size(0)
        
        avg_tr_loss = accumulated_tr_loss / (batch+1)
        avg_tr_accuracy = 100*accumulated_tr_accuracy / (batch+1)

        # Backward path (Gradient)
        vit_optimizer.zero_grad()
        loss.backward()

        # Optimizer(Adam) Step
        vit_optimizer.step()

        loop.set_description(f"Epoch [{epoch+1}/{vit_epochs}]")
        loop.set_postfix(loss=avg_tr_loss, acc=avg_tr_accuracy)
    
    tr_loss_history.append(avg_tr_loss)
    tr_accuracy_history.append(avg_tr_accuracy)

    scheduler.step()


# Validation Loop
    vit_model.eval()

    with torch.no_grad():

        for batch, (_, inputs, targets) in enumerate(Im_val_loader):

            # Put inputs and labels in device cuda
            val_images = inputs.to(device)
            val_labels = targets.to(device)

            val_outputs = vit_model(val_images).squeeze(1)

            loss = criterion(val_outputs,val_labels)

            accumulated_val_loss += loss.item()
            accumulated_val_accuracy += (val_labels == val_outputs).sum().item()/val_labels.size(0)

            avg_val_loss = accumulated_val_loss / (batch+1)
            avg_val_accuracy = 100*accumulated_val_accuracy / (batch+1)

        val_loss_history.append(avg_val_loss)
        val_accuracy_history.append(avg_val_accuracy)

        print(f'Validation loss: {avg_val_loss:.2f} | Validation accuracy: {avg_val_accuracy:.2f}')
        print("-------------------------------------------------------------------------------")

In [ ]:
fig, axs = plt.subplots(2, 1)

axs[0].plot(range(len(tr_loss_history )), tr_loss_history , label='Train Loss')
axs[1].plot(range(len(tr_accuracy_history)), tr_accuracy_history, label='Validation Loss')
axs[0].plot(range(len(val_loss_history)), val_loss_history, label='Train Accuracy')
axs[1].plot(range(len(val_accuracy_history)), val_accuracy_history, label='Validation Accuracy')
plt.show()